In [1]:
import re
from pathlib import Path

from tensorboard.backend.event_processing import event_accumulator
import pandas as pd

def analy(ds):
    def parse_tensorboard(path, scalars):
        """returns a dictionary of pandas dataframes for each requested scalar"""
        ea = event_accumulator.EventAccumulator(
            path,
            size_guidance={event_accumulator.SCALARS: 0},
        )
        _absorb_print = ea.Reload()
        # make sure the scalars are in the event accumulator tags
        assert all(
            s in ea.Tags()["scalars"] for s in scalars
        ), "some scalars were not found in the event accumulator"
        return {k: pd.DataFrame(ea.Scalars(k)) for k in scalars}

    # all dirs like data/fomo/output/CoPrompt_
    dirs = list(Path('data/fomo/output').glob('CoPrompt_*'))
    rs = []
    rsa = []
    for dir in dirs:
        seed1_train = dir / f'train_base/{ds}/shots_16/CoPrompt/coprompt/seed1/'
        tensorboard = seed1_train / 'tensorboard'
        clog = seed1_train / 'clog.txt'
        
        if not clog.exists():
            continue
            
        with open(clog, 'r') as f:
            lines = f.readlines()
            if len(lines) < 2:
                continue
            p = lines[-1][4:-5]
        
        # read from tensorboard)
        try:
            df = parse_tensorboard(tensorboard, ['test/accuracy'])
            test_acc = df['test/accuracy'].iloc[-1]['value']
        except Exception as e:
            continue
            
        rs.append((test_acc, p, str(dir)))
        
        seed1_test = dir / f'test_new/{ds}/shots_16/CoPrompt/seed1/'
        out = seed1_test / 'output.log'
# => result
# * total: 1,410
# * correct: 1,070
# * accuracy: 75.9%
# * error: 24.1%
# * macro_f1: 70.8%
        
        try:
            # use regex to match format above
            acc = re.search(r'accuracy: ([0-9.]+)%', out.read_text()).group(1)
            acc = float(acc)
            rsa.append((test_acc, p, acc, str(dir)))
        except Exception as e:
            pass
        
        
        
        
    print(len(rs), len(rsa))
    for test_acc, p, m in list(sorted(rs, reverse=True))[:5]:
        print(f"[{test_acc}], {p}")
        print(m)

    print('---')
    for test_acc, p, acc, m in list(sorted(rsa, reverse=True, key=lambda x:  x[0] + x[2]))[:5]:
        print(f"[{test_acc}], [{acc}], {p}")
        print(m)
    # 
    #%%
        

In [2]:
analy('ucf101')

2024-05-12 16:07:03.953061: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-12 16:07:03.998776: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-12 16:07:04.750659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


123 47
[77.86969757080078], casually whidoing  
data/fomo/output/CoPrompt_m0.9_wstd0.001_nctxt3_maxepoch16
[77.24922180175781], katelyn davis doing "#  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt4_maxepoch16
[76.9389877319336], casually trying doing  
data/fomo/output/CoPrompt_m0.9_wstd0.012_nctxt3_maxepoch16
[76.8355712890625], freaking iit alex egan #  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt5_maxepoch8
[76.8355712890625], daimhkinternational chan can responsibly "  
data/fomo/output/CoPrompt_m0.0_wstd0.008_nctxt7_maxepoch16
---
[75.64633178710938], [78.0], inspirational  
data/fomo/output/CoPrompt_m0.9_wstd0.012_nctxt1_maxepoch16
[77.86969757080078], [75.2], casually whidoing  
data/fomo/output/CoPrompt_m0.9_wstd0.001_nctxt3_maxepoch16
[76.68045806884766], [75.9], jing altman attempting  
data/fomo/output/CoPrompt_m0.0_wstd0.012_nctxt3_maxepoch16
[76.9389877319336], [74.7], casually trying doing  
data/fomo/output/CoPrompt_m0.9_wstd0.012_nctxt3_maxepoch16
[76.783866882324

In [3]:
analy('eurosat')

48 48
[90.61904907226562], aturegodly novascotia cornwall  
data/fomo/output/CoPrompt_m0.0_wstd0.08_nctxt4_maxepoch16
[89.54762268066406], organic career ativesized  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt4_maxepoch16
[89.21428680419922], increased underdog gihc  
data/fomo/output/CoPrompt_m0.0_wstd0.012_nctxt4_maxepoch16
[86.64286041259766], coronado rugnovascotia  
data/fomo/output/CoPrompt_m0.0_wstd0.08_nctxt3_maxepoch16
[85.88095092773438], pastoral  
data/fomo/output/CoPrompt_m0.9_wstd0.08_nctxt1_maxepoch16
---
[89.21428680419922], [79.9], increased underdog gihc  
data/fomo/output/CoPrompt_m0.0_wstd0.012_nctxt4_maxepoch16
[89.54762268066406], [75.5], organic career ativesized  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt4_maxepoch16
[77.83333587646484], [80.6], # canadiens  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt2_maxepoch16
[80.33333587646484], [76.3], taco lue novascotia  
data/fomo/output/CoPrompt_m0.0_wstd0.08_nctxt3_maxepoch8
[74.88095092773438], [81.3], r

In [4]:
analy('oxford_flowers')

48 48
[77.0180435180664], flowering stripes  
data/fomo/output/CoPrompt_m0.0_wstd0.012_nctxt2_maxepoch16
[76.82810974121094], wonderful flowering varieties –  
data/fomo/output/CoPrompt_m0.9_wstd0.001_nctxt4_maxepoch16
[76.73314666748047], flowerflowering wonderful  
data/fomo/output/CoPrompt_m0.0_wstd0.012_nctxt3_maxepoch16
[76.63817596435547], attracting flowering gardening –  
data/fomo/output/CoPrompt_m0.0_wstd0.012_nctxt4_maxepoch16
[76.06837463378906], autumn flowers  
data/fomo/output/CoPrompt_m0.0_wstd0.08_nctxt2_maxepoch16
---
[74.54890441894531], [78.3], flowering flower  
data/fomo/output/CoPrompt_m0.9_wstd0.08_nctxt2_maxepoch16
[75.68850708007812], [76.0], flower  
data/fomo/output/CoPrompt_m0.9_wstd0.012_nctxt1_maxepoch16
[75.30863952636719], [76.3], flowering  
data/fomo/output/CoPrompt_m0.9_wstd0.012_nctxt1_maxepoch8
[75.59354400634766], [75.9], perennial wonderful scented :-  
data/fomo/output/CoPrompt_m0.0_wstd0.012_nctxt4_maxepoch8
[75.40361022949219], [75.7], pollina

In [5]:
analy('dtd') # 

47 47
[63.657405853271484], beautiful abstract  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt2_maxepoch16
[62.96296310424805], beautiful abstract  
data/fomo/output/CoPrompt_m0.0_wstd0.012_nctxt2_maxepoch16
[62.73147964477539], three ici  
data/fomo/output/CoPrompt_m0.0_wstd0.08_nctxt2_maxepoch16
[62.61574172973633], vibrant vibrant  
data/fomo/output/CoPrompt_m0.9_wstd0.08_nctxt2_maxepoch16
[62.38425827026367], textures abstract  
data/fomo/output/CoPrompt_m0.9_wstd0.001_nctxt2_maxepoch16
---
[62.96296310424805], [62.8], beautiful abstract  
data/fomo/output/CoPrompt_m0.0_wstd0.012_nctxt2_maxepoch16
[62.03703689575195], [61.8], cropped  
data/fomo/output/CoPrompt_m0.9_wstd0.001_nctxt1_maxepoch16
[63.657405853271484], [58.9], beautiful abstract  
data/fomo/output/CoPrompt_m0.0_wstd0.001_nctxt2_maxepoch16
[62.38425827026367], [59.3], textures abstract  
data/fomo/output/CoPrompt_m0.9_wstd0.001_nctxt2_maxepoch16
[58.33333206176758], [62.8], deeply ically exploring  
data/fomo/output/Co

In [6]:
analy('fgvc_aircraft') # 

0 0
---


In [7]:
analy('caltech101') # 

0 0
---


In [8]:
analy('oxford_pets') # 

0 0
---


In [9]:
analy('stanford_cars') # 

0 0
---


In [10]:
analy('food101') # 

0 0
---


In [11]:
analy('sun397') # 

0 0
---


In [12]:
analy('imagenet') # 

0 0
---
